In [1]:
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import glob
import datetime
import time
import sys
import os
import tables
import io
import gzip

pd.set_option('display.max_columns', None)

In [3]:
# !pip install tables
# !pip install bs4

In [4]:
# Path to the Million Song Dataset subset (uncompressed) - change to the location on your laptop
# Cannot store this on github as it is too large
msd_subset_path = '../../MSD_data/MillionSongSubset/'

# Keep these - folders match the structure of the uncompressed file
msd_subset_data_path = os.path.join(msd_subset_path, 'data')
msd_subset_addf_path = os.path.join(msd_subset_path, 'AdditionalFiles')

In [5]:
# # Load previously saved file
subset_full = pd.read_pickle(msd_subset_path+'subset_full.pkl')
# keyData = pd.read_pickle(msd_subset_path+'keydata.pkl')
# keyDataAndH = pd.read_pickle(msd_subset_path+'keydata_and_hotness.pkl')

In [6]:
keyData = subset_full[["track_id", "song_id", "song_hotttnesss"]]
keyDataAndH = subset_full[["track_id", "song_id", "song_hotttnesss"]].dropna()
keyData.to_pickle(msd_subset_path+'keydata.pkl')
keyDataAndH.to_pickle(msd_subset_path+'keydata_and_hotness.pkl')

In [7]:
# Make sure sid_mismatches.txt is gzipped before running this!
def prepSidMismatch() :
    with io.TextIOWrapper(gzip.open('../../MSD_data/sid_mismatches_prep.txt.gz', "w")) as preproc:
        with io.TextIOWrapper(gzip.open('../../MSD_data/sid_mismatches.txt.gz', "r")) as src:
            preproc.write('song_id,track_id\n')
            for line in src:
                line = ','.join([w.replace('<', '').replace('>', '') for w in line.split(' ')[1:3]]) + '\n'
                preproc.write(line)

In [8]:
prepSidMismatch()

In [9]:
mism_data = pd.read_csv('../../MSD_data/sid_mismatches_prep.txt.gz', compression='gzip')

SSD data is at https://drive.google.com/open?id=1p0_2sURyJmFIlMf0fk9SjtWc2CT7s-Gu

In [10]:
rh_data = pd.read_csv('../../MSD_data/msd-rh.csv.gz', header=None, compression='gzip', escapechar='\'')
rh_data[60]=rh_data[60].str.replace(',','')
rh_data.rename(columns={60: 'track_id'}, inplace=True)
rh_hotness = keyDataAndH.merge(rh_data, how = 'left', on = 'track_id')
rh_hotness.to_pickle(msd_subset_path+'rh_and_hotness.pkl')
rh_hotness = pd.read_pickle(msd_subset_path+'rh_and_hotness.pkl')
keys = ['track_id', 'song_id']
idx1 = set(rh_hotness.set_index(keys).index)
idx2 = set(mism_data.set_index(keys).index)
df_clean=pd.DataFrame(list(idx1 - idx2), columns=keys)
rh_hotness_clean = rh_hotness.merge(df_clean, how = 'left', on = keys)
rh_hotness_clean.to_pickle(msd_subset_path+'rh_and_hotness_clean.pkl')
ssd_data = pd.read_csv('../../MSD_data/msd-ssd-v1.0.csv.gz', header=None, compression='gzip', escapechar='\'')
ssd_data[168]=ssd_data[168].str.replace(',','')
ssd_data.rename(columns={168: 'track_id'}, inplace=True)
ssd_hotness = keyDataAndH.merge(ssd_data, how = 'left', on = 'track_id')
ssd_hotness.to_pickle(msd_subset_path+'ssd_and_hotness.pkl')
idx1 = set(ssd_hotness.set_index(keys).index)
df_clean=pd.DataFrame(list(idx1 - idx2), columns=keys)
ssd_hotness_clean = ssd_hotness.merge(df_clean, how = 'left', on = keys)
ssd_hotness_clean.to_pickle(msd_subset_path+'ssd_and_hotness_clean.pkl')
# rh_hotness_clean = pd.read_pickle(msd_subset_path+'rh_and_hotness_clean.pkl')
# ssd_hotness_clean = pd.read_pickle(msd_subset_path+'ssd_and_hotness_clean.pkl')

In [15]:
ssd_data = pd.read_csv('../../MSD_data/msd-ssd-v1.0.csv.gz', header=None, compression='gzip', escapechar='\'')
ssd_data[168]=ssd_data[168].str.replace(',','')
ssd_data.rename(columns={168: 'track_id'}, inplace=True)
ssd_hotness = keyDataAndH.merge(ssd_data, how = 'left', on = 'track_id')
ssd_hotness.to_pickle(msd_subset_path+'ssd_and_hotness.pkl')
idx1 = set(ssd_hotness.set_index(keys).index)
df_clean=pd.DataFrame(list(idx1 - idx2), columns=keys)
ssd_hotness_clean = ssd_hotness.merge(df_clean, how = 'left', on = keys)
ssd_hotness_clean.to_pickle(msd_subset_path+'ssd_and_hotness_clean.pkl')
ssd_hotness_clean.shape

(5648, 171)